<a href="https://colab.research.google.com/github/yumi-jun/Data_Analysis/blob/main/artificial_neural_network_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
#텐서 플로우 버
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [7]:
dataset = pd.read_csv('Churn_Modelling.csv')
# 이름과 번호를 제외한 데이터
X = dataset.iloc[:, 3:-1].values
#이탈률
y = dataset.iloc[:, -1].values

In [8]:
print(X)


[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [9]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [10]:
# LabelEncoder 을 sklearn.preprocessing 모듈에서 가져오고,
# 범주형 데이터를 라벨 데이터 로 변환한다.
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# 성별 데이터 female, male 을 0,1 로 변환
X[:, 2] = le.fit_transform(X[:, 2])

In [11]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

"지역" 열을 원-핫 코딩을 해보자 !

원핫 코딩이란?
범주형 데이터를 이진 벡터로 변환하는

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# 적용하고 싶은 열 -> 두번째 열, 인덱스 1번째
# OneHotEncoder 가 해당 열을 인코딩하면서 새로운 열들을 추가하고,
# 인코딩 된 열들을 가장 앞으로 배치한다.
# ColumnTransformer 는 변환된 열을 기본적으로 앞쪽에 위치시킨다.

#프랑스는 1,0,0, 독일 0,0,1 이렇게 인코딩 된다!
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)


[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [21]:
# 데이터를 무작위로 학습 세트와 테스트 세트로 나눈다.
from sklearn.model_selection import train_test_split
# 학습 데이터는 80, 훈련 데이터는 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [24]:
# 분할된 데이터의 크기 출력
print(X.shape)
print("X_train 크기:", X_train.shape)
print("X_test 크기:", X_test.shape)
print("y_train 크기:", y_train.shape)
print("y_test 크기:", y_test.shape)

(10000, 12)
X_train 크기: (8000, 12)
X_test 크기: (2000, 12)
y_train 크기: (8000,)
y_test 크기: (2000,)


### Feature Scaling

In [25]:
# 데이터픞 표준화하는 과정
# 데이터의 평균을 0, 표준편차를 1로
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# 데이터의 평균과 표준편차를 계산, 학습용데이터에 대해 사용
X_train = sc.fit_transform(X_train)
# 이미 계산된 평균과 표준편차를 사용해 데이터를 변환
# 테스트 데이터에 대해 사용한다.
X_test = sc.transform(X_test)


## Part 2 - Building the ANN
## 인공신경망을 만들어보자 - 구조 정의

### Initializing the ANN


Adding the second hidden layer



In [13]:
ann = tf.keras.models.Sequential()

### Adding the second hidden layer

In [14]:
# ann.add : 텐서 플로우를 사용하여, 인공 신경망의 레이어를 추가하는 것
# tf.keras.layers.Dense 밀집 레이어를 의미, 레이어의 뉴런 노드 수, 활성화 함수 relu : ReLu 활성화 함수
# 입력이 0보다 작으면, 0출력, 0보다 크면 입력 값을 그대로 출력
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))


### Adding the output layer

In [15]:
# 마지막 출력층 값 ->출력증 값이 이진 변수로 1로 변경
# 만약 출력층 값이, abc 와 같이 이진 변수가 아닐 경우에는 1 이 아닌 3 으로 ,,
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


## Part 3 - Training the ANN
학습 준

### Compiling the ANN

In [17]:
# 모델이 학습할 때 , 가중치를 조정하는 방법 optimizer , adam 은 최적화 알고리즘 : 학습을 더 빠르고 효율적으로
# loss: 모델이 예측한 값과 실제 값의 차이를 계산, binary_crossentropy : 이진 분류 문제에 적합한 손실 함
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


### Training the ANN on the Training set

In [27]:
# 학습할 입력데이터, 학습용 정답 레이블, 모델이 한 번에 학습할 데이터의 샘플 수 :32 , 32개의 샘플을 한번에 처리
# 에포크 수 : 전체 데이터 셋에 대해 학습을 반복하는 횟
ann.fit(X_train ,y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 0.5556 - accuracy: 0.7381
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4721 - accuracy: 0.7959
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4464 - accuracy: 0.7970
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4302 - accuracy: 0.8065
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.4170 - accuracy: 0.8145
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.4039 - accuracy: 0.8265
Epoch 7/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3903 - accuracy: 0.8339
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3786 - accuracy: 0.8429
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3702 - accuracy: 0.8460
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3634 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [30]:
# 예측 메소드의 입력값은 2d 배열이여야 한다.
# sc.transform : 정규화하는 과정
# 마지막 ann 을 만들때의 함수가 시그노이드 함수 -> 확률 로 값을 출력해
# 0.5 확률보다 아래니까 떠날 확률은 낮다!!!
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))


1/1 [==============================] - 0s 20ms/step
[[0.3896976]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [36]:
# 테스트 데이터 x_text 에 대해 모델이 예측을 수행
# y_pred 에 모델이 예측한 확률값을 담고 있다.
y_pred = ann.predict(X_test)
print(y_pred) # 각 행의 예측값들
y_pred = (y_pred > 0.5) # 예측값을 0 과 1로 바꿈 이진화

# y_pred 와 y_test 를 나란히 결합하여 비교
# y_pred 는 방금 만든 딥러닝 모델로 예측한 값이고
# y_test 는 이미 데이터에 주어진 실제 값.
# reshape 로 각각의 열 벡터로 변환
# np.concatenate 는 두 열 벡터를 옆으로 결합
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))
# 1은 은행을 떠난 것, 0은 떠나지 않은

63/63 [==============================] - 0s 1ms/step
[[0.28022423]
 [0.33390072]
 [0.17737262]
 ...
 [0.11820526]
 [0.17131951]
 [0.16235702]]
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [37]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1502   93]
 [ 188  217]]


0.8595